# sgc test 2



In [0]:
%sh nvidia-smi

In [0]:
import logging
logging.basicConfig(level=logging.INFO)
# In your notebook
import time
import sys
import os

In [0]:
import yaml
cfg = yaml.safe_load(open('config.yaml'))
compression = "nvImage_HTJ2K_progressive"
input_path = cfg.get("input_path")
output_path = cfg.get("output_path").replace("{compression}", f"{compression}")
table = "main.pixels_solacc.object_catalog"
experiment_log_dir = "/Volumes/douglas_moore/mlflow/experiments"
folder_limit = 4
num_gpus = 2
CODE_FOLDER = '/Workspace/Users/douglas.moore@databricks.com/pixels-jpeg2000/notebooks/transcoding'
input_path, output_path

#set GPU logging


## Pull dir paths to compress

In [0]:
import pandas as pd
import numpy as np

#
## Chunk up dirs
#
from pyspark.sql.functions import col, split, size, slice
df = spark.read.table("hls_radiology.tcia.object_catalog")
from pyspark.sql.functions import col, split, size, slice, concat
from pyspark.sql.functions import array_join
df = df.withColumn('dirs', array_join(slice(split(col('path'), '/'), 1, size(split(col('path'), '/')) - 1), '/'))
df = df.withColumn('dir', split(col('dirs'),':')[1])
df = df.select('dir').distinct().orderBy('dir').limit(folder_limit)
pdf = df.toPandas()
chunks = np.array_split(pdf, num_gpus)


In [0]:

from serverless_gpu import distributed
from serverless_gpu import runtime as rt

@distributed(gpus=num_gpus, gpu_type='a10', remote=True)
def multi_a10(chunks, output_path:str):

  #init
  import logging
  import sys
  import os

  # Initialization (per worker basis)
  logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
  logger = logging.getLogger()
  sys.path.append(CODE_FOLDER)
  from convert_htj2k_p2 import transcode_dicom_to_htj2k
  os.environ['PYNVIMGCODEC_VERBOSITY'] = '5'

  logger.info(chunks[rt.get_global_rank()])
  output_dirs=[]

  #
  ## Loop through each chunk
  #
  for dir in chunks[rt.get_global_rank()]['dir']:
    logger.info(dir)
    try:
      output_dir = transcode_dicom_to_htj2k(input_dir = dir, output_dir=output_path)
      output_dirs.append(output_dir)
    except Exception as e:
      logger.error("Exception caught processing {dir}: %s", e, exc_info=True)
  
  return rt.get_global_rank(), rt.get_world_size(), output_dirs

multi_a10.distributed(chunks=chunks, output_path=output_path) # returns a list, one element per GPU
#multi_a10(chunks, output_path)